In [2]:
import pandas as pd
import re
import csv

### Outlier decetion of internal data

In [ ]:
pdObj = pd.read_json('../data/curated/domain.json', orient='index')
pdObj.to_csv('streaming_test.csv', index=True)

In [ ]:
# Converte json document to csv.
pdObj = pd.read_json('domain.json', orient='index')
pdObj.to_csv('streaming_test.csv', index=True)

In [ ]:
# Get the cost
df = pd.read_csv('streaming_test.csv')
df['cost'] = df['cost_text']
for i in range(len(df)):
    df['cost'].iloc[i] = re.findall(r'\d+\.?\d*',df['cost_text'].iloc[i])

In [ ]:
# Delete missing values
delete_list = []
df_new = df.copy()
for i in range(len(df)):
    if len(df['cost'].iloc[i]) == 0:
        delete_list.append(i)
df_new = df_new.drop(df_new.index[delete_list])

In [ ]:
# Transform the data type of cost into float.
for i in range(len(df_new)):
    df_new['cost'].iloc[i] = float(df_new['cost'].iloc[i][0])
    
df_new['cost'] = df_new['cost'].astype('float')

In [ ]:
# Delete the extreme values
delete_list = []
k1 = 0.8
k2 = 0.2
df_new2 = df_new.copy()
for i in range(len(df_new2)):
    if df_new2['cost'].iloc[i] < df_new2['cost'].mean()*(1-k1) or df_new2['cost'].iloc[i] > df_new2['cost'].mean()*(1+k2):
        delete_list.append(i)
df_new2 = df_new2.drop(df_new2.index[delete_list])
df_new2.describe()

In [149]:
df = pd.read_csv("../data/raw/2021_GCP_SA2_for_VIC_short-header/2021 Census GCP Statistical Area 2 for VIC/2021Census_G33_VIC_SA2.csv")
df.head()

,SA2_CODE_2021,Neg_Nil_inc_fam_households,Neg_Nil_inc_Non_fam_househlds,Negative_Nil_income_Tot,HI_1_149_Family_households,HI_1_149_Non_family_hseholds,HI_1_149_Tot,HI_150_299_Family_households,HI_150_299_Non_fam_househlds,HI_150_299_Tot,...,HI_4000_more_Tot,Part_inc_stated_Fam_household,Part_inc_std_Non_fam_hhds,Partial_income_stated_Tot,All_incomes_ns_Famly_househld,All_inc_ns_Non_famly_househld,All_incomes_not_stated_Tot,Tot_Family_households,Tot_Non_family_households,Tot_Tot
0,201011001,28,34,58,14,16,31,13,26,38,...,618,284,24,306,30,37,68,4481,1275,5763
1,201011002,18,57,75,9,21,32,15,60,72,...,682,164,34,195,28,35,62,3032,2006,5036
2,201011005,14,24,38,14,8,14,6,19,24,...,328,105,7,113,13,14,28,1989,534,2522
3,201011006,21,27,47,6,11,24,15,20,34,...,181,150,16,166,37,39,70,2887,981,3870
4,201011007,11,7,18,3,4,10,7,10,11,...,151,98,3,102,14,3,18,1174,222,1395


In [150]:
cols = [col for col in df.columns if 'Tot' in col and 'Neg' not in col and 'stated' not in col]

In [151]:
df1 = df[cols].drop(['Tot_Family_households','Tot_Non_family_households','Tot_Tot'],axis=1)
df1['SA2_CODE21'] = df['SA2_CODE_2021']
df1.head()

,HI_1_149_Tot,HI_150_299_Tot,HI_300_399_Tot,HI_400_499_Tot,HI_500_649_Tot,HI_650_799_Tot,HI_800_999_Tot,HI_1000_1249_Tot,HI_1250_1499_Tot,HI_1500_1749_Tot,HI_1750_1999_Tot,HI_2000_2499_Tot,HI_2500_2999_Tot,HI_3000_3499_Tot,HI_3500_3999_Tot,HI_4000_more_Tot,SA2_CODE21
0,31,38,81,232,211,284,327,418,384,344,349,702,529,485,297,618,201011001
1,32,72,119,324,282,305,331,379,383,316,253,484,280,275,181,682,201011002
2,14,24,35,108,101,145,138,182,176,128,129,291,222,195,120,328,201011005
3,24,34,64,213,143,242,259,328,340,261,309,548,323,218,104,181,201011006
4,10,11,21,54,34,78,73,72,97,69,84,185,147,112,65,151,201011007


In [152]:
df1.columns[0]

'HI_1_149_Tot'

In [153]:
avg = 0
df1['total_amount'] = 0
df1['family_amount'] = 0
for i in range(len(df1.columns)-3):
    if(i != 15):
        low = int(re.findall(r'[0-9]+', df1.columns[i])[0])
        upper = int(re.findall(r'[0-9]+', df1.columns[i])[1])
        avg = low + ((upper - low)/2)
    else:
        avg = 5500
    df1['total_amount'] = df1['total_amount'] + avg * df1.iloc[:, i]
    df1['family_amount'] = df1['family_amount'] + df1.iloc[:, i]
df1['avg'] = df1['total_amount']/df1['family_amount']

In [154]:
df1.head()

,HI_1_149_Tot,HI_150_299_Tot,HI_300_399_Tot,HI_400_499_Tot,HI_500_649_Tot,HI_650_799_Tot,HI_800_999_Tot,HI_1000_1249_Tot,HI_1250_1499_Tot,HI_1500_1749_Tot,HI_1750_1999_Tot,HI_2000_2499_Tot,HI_2500_2999_Tot,HI_3000_3499_Tot,HI_3500_3999_Tot,HI_4000_more_Tot,SA2_CODE21,total_amount,family_amount,avg
0,31,38,81,232,211,284,327,418,384,344,349,702,529,485,297,618,201011001,12097684.5,5330,2269.734428
1,32,72,119,324,282,305,331,379,383,316,253,484,280,275,181,682,201011002,10008608.0,4698,2130.397616
2,14,24,35,108,101,145,138,182,176,128,129,291,222,195,120,328,201011005,5403328.0,2336,2313.068493
3,24,34,64,213,143,242,259,328,340,261,309,548,323,218,104,181,201011006,6672032.0,3591,1857.987190
4,10,11,21,54,34,78,73,72,97,69,84,185,147,112,65,151,201011007,2918874.0,1263,2311.064133


In [155]:
df2 = df1[['SA2_CODE21','avg']]
df2

,SA2_CODE21,avg
0,201011001,2269.734428
1,201011002,2130.397616
2,201011005,2313.068493
3,201011006,1857.987190
4,201011007,2311.064133
...,...,...
519,217041478,1905.366053
520,217041479,1850.573197
521,217041480,1823.943042
522,297979799,NaN


In [156]:
df2.to_csv('../data/curated/income.csv',index=False)

### Locality to SA2 Coding Index

In [304]:
index = pd.read_csv("../data/raw/australian_postcodes.csv")
sa2_2021 = pd.read_csv("../data/raw/CG_SA2_2016_SA2_2021.csv")

In [ ]:
len(set(sa2_2021['SA2_MAINCODE_2016']))

2311

In [330]:
index_2021 = sa2_2021.merge(index, on='SA2_MAINCODE_2016')
index_2021 = index_2021[index_2021['state'] == 'VIC']

In [331]:
index2 = index_2021[['SA2_MAINCODE_2016', 'SA2_NAME_2016_x', 'SA2_CODE_2021', 'SA2_NAME_2021','postcode','locality','Lat_precise','Long_precise']]
index2 = index2.drop_duplicates(subset=['postcode'])

In [332]:
len(set(index2['postcode']))

747

In [336]:
postcode = pd.read_csv('../data/raw/Australian-Postcode-Data-master/au_postcodes.csv')
code_vic = postcode[postcode['state_code'] == 'VIC']
code_vic = code_vic[postcode['accuracy'] == 4]
code_vic = code_vic.drop_duplicates(subset=['postcode'])

/var/folders/xx/831dtzvs55b0c08vg3m5yhf40000gn/T/ipykernel_52832/3115002455.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  code_vic = code_vic[postcode['accuracy'] == 4]


In [337]:
index_vic = code_vic[['postcode','latitude','longitude']].merge(index2, on='postcode')

In [338]:
index_vic
index_vic.to_csv('../data/curated/postcode_to_sa2.csv',index=False)

In [339]:
index_vic.count()

postcode             717
latitude             717
longitude            717
SA2_MAINCODE_2016    717
SA2_NAME_2016_x      717
SA2_CODE_2021        717
SA2_NAME_2021        717
locality             717
Lat_precise          717
Long_precise         717
dtype: int64

In [162]:
domain = pd.read_csv("../data/curated/domain.csv")

In [235]:
domain.loc[domain['postcode'] == 3337, 'suburb_sold'] = 99
domain.loc[domain['postcode'] == 3337, 'suburb_avg_day'] = 43
domain.loc[domain['postcode'] == 3337, 'suburb_population'] = 17589
domain.loc[domain['postcode'] == 3337, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3337, 'suburb_ower'] = '74%'
domain.loc[domain['postcode'] == 3337, 'suburb_renter'] = '26%'
domain.loc[domain['postcode'] == 3337, 'suburb_family'] = '46%'
domain.loc[domain['postcode'] == 3337, 'suburb_single'] = '54%'


In [259]:
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_sold'].isnull()), 'suburb_sold'] = 97
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_avg_day'].isnull()),'suburb_avg_day'] = 14
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_population'].isnull()),'suburb_population'] = 8343
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_age'].isnull()),'suburb_age'] = '60+'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_ower'].isnull()), 'suburb_ower'] = '80%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_renter'].isnull()),'suburb_renter'] = '20%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_family'].isnull()),'suburb_family'] = '51%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_single'].isnull()),'suburb_single'] = '49%'


In [251]:
domain[domain['postcode'] == 3350].count()

Unnamed: 0           166
Unnamed: 0.1         166
name                 166
cost_text            166
type                 166
school_title         166
school_distance      166
suburb_sold          166
suburb_avg_day       166
suburb_population    166
suburb_age           166
suburb_ower          166
suburb_renter        166
suburb_family        166
suburb_single        166
coordinates          166
rooms                166
desc                 166
postcode             166
cost                 166
dtype: int64

In [273]:
domain.loc[(domain['postcode'] == 3000)&(domain['suburb_population'].isnull())]

,Unnamed: 0,Unnamed: 0.1,name,cost_text,type,school_title,school_distance,suburb_sold,suburb_avg_day,suburb_population,suburb_age,suburb_ower,suburb_renter,suburb_family,suburb_single,coordinates,rooms,desc,postcode,cost


In [270]:
post_n = set(domain[domain['suburb_population'].isnull()]['postcode'])

In [300]:
for post in post_n:
    if post not in [3004,3195]:
        domain =  domain.drop(domain[(domain['postcode'] == post)&(domain['suburb_population'].isnull())].index)

In [301]:
domain[domain['suburb_population'].isnull()]['postcode'].value_counts()

3004    57
3195    35
Name: postcode, dtype: int64

In [303]:
domain.to_csv('../data/curated/domain_2.csv',index = False)

In [310]:
domain2 = pd.read_csv("../data/curated/domain_4.csv")

In [317]:
domain3 = pd.read_csv("../data/curated/df_property_result.csv")

Unnamed: 0           9150
URL                  9150
name                 9150
type                 9150
school_title         9149
school_distance      9149
suburb_sold          9093
suburb_avg_day       9093
suburb_population    9093
suburb_age           9093
suburb_ower          9093
suburb_renter        9093
suburb_family        9093
suburb_single        9093
coordinates          9150
postcode             9150
cost                 9150
Beds                 9150
Bath                 9150
Parking              9150
dtype: int64